# Оценка кликбейтности и создание нейроновостей

Здесь можно посмотреть, как работает код из статьи «Компьютер учит срамоту»: оценить желтизну придуманного вами заголовка и нагенерировать нейрозаголовков (и нейротекстов для них)

## Кликбейтность

In [1]:
#@markdown ### Подготовка оценки кликбейтности
#@markdown ← Нажмите '▷' слева, чтобы подготовить модель оценки
 
!pip install -qqq sentence-transformers 1>/dev/null 2>/dev/null
!wget -q "https://github.com/sysblok/neuroclickbait/raw/master/vectors/clickbait_vector.npy" -O clickbait_vector.npy
!wget -q "https://github.com/sysblok/neuroclickbait/raw/master/vectors/cluster_centers.npy" -O cluster_centers.npy


try:
  from IPython.utils import io
  import numpy as np
 
  from math import sqrt
  from scipy.stats import norm
  from sentence_transformers import SentenceTransformer
  from IPython.core.display import display, HTML
 
 
  def printhtml(instr):
    display(instr)
 
  def uniform_transform(x, mean, var, interval=(0, 50)):
    i = interval
    std = sqrt(var)
    x -= mean
    return -i[0] + i[1] * norm.cdf(x * sqrt(1/var), 0, std)
 
  def get_cluster_distance(embedding, cluster_centers):
    return min([np.linalg.norm(embedding - center) for center in cluster_centers])
 
  def get_clickbait_likeness(embedding, clickbait_vector):
    return np.linalg.norm(embedding - clickbait_vector)
 
  def get_embedding_clickbait_score(embedding, cluster_centers, clickbait_vector):
    cluster_closeness_score = 50 - uniform_transform(get_cluster_distance(embedding, cluster_centers), 4.379236, 0.40359718)
    clickbait_likeness_score = 50 - uniform_transform(get_clickbait_likeness(embedding, clickbait_vector), 4.7919064, 0.36316898)
    return cluster_closeness_score + clickbait_likeness_score
 
  def get_assessment_for_score(i):
    if i == 0: return '<div style="display: inline; color: #5a8150; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: Абсолютно пресный. Посетители сайтов про секреты знаменитостей на такое точно не кликнут.</div>'
    elif i < 10: return '<div style="display: inline; color: #688749; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: Вообще не жёлтый.</b> Такой точно не поставят под баннером.</div>'
    elif i < 20: return '<div style="display: inline; color: #758d41; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: Совсем не жёлтый.</b> Такой не вызывает жёлания скорее нажать и читать...</div>'
    elif i < 30: return '<div style="display: inline; color: #83933a; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: Не жёлтый.</b> Неплохая попытка, но всё-таки не то.</div>'
    elif i < 40: return '<div style="display: inline; color: #919933; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: Начинает желтеть.</b> Возможно, стоит добавить "ШОК!" или "Жмите по ссылке..."?</div>'
    elif i < 50: return '<div style="display: inline; color: #9e9f2c; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: Почти жёлтый.</b> Это близко! Но всё же недотягивает.</div>'
    elif i < 60: return '<div style="display: inline; color: #aca624; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: Довольно-таки жёлтый.</b> Такой уже можно увидеть на трешовом баннере.</div>'
    elif i < 70: return '<div style="display: inline; color: #b9ac1d; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: Вполне себе жёлтый.</b> Качественный, сочный, как лимон.</div>'
    elif i < 80: return '<div style="display: inline; color: #c7b216; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: Жёлтый.</b> Отличный заголовок! Клики от посетителей сайтов о секретах здоровья вам обеспечены.</div>'
    elif i < 90: return '<div style="display: inline; color: #d5b80f; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: Очень жёлтый.</b> Самое то. Возможно, где-нибудь на сайте о волшебном обогащении такой даже висит.</div>'
    elif i < 100: return '<div style="display: inline; color: #e2be07; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: Максимально жёлтый!</b> У вас талант! Не задумывались о трудоустройстве?</div>'
    return '<div style="display: inline; color: #f0c400; font-size: 18px;"><b><div style="display: inline; font-size: 24px;">{}</div>: МАКСИМАЛЬНО жёлтый!</b> Вы точно не автор жёлтых баннеров?</div>'
 
  def clickbaitness_score(prompt, model, cluster_centers, clickbait_vector):
    return get_embedding_clickbait_score(model.encode(prompt), cluster_centers=cluster_centers, clickbait_vector=clickbait_vector)
 
  def prettyprint_clickbait_score(prompt, model, cluster_centers, clickbait_vector):
    clickbait_score = clickbaitness_score(prompt, model=model, cluster_centers=cluster_centers, clickbait_vector=clickbait_vector)
    if clickbait_score < 0:
      clickbait_score = 0
    elif clickbait_score > 100:
      clickbait_score = 100
    clickbait_score = int(round(clickbait_score))
    output = get_assessment_for_score(clickbait_score)
    output = output.format(str(clickbait_score))
    printhtml(HTML('<div style="display: inline; font-size: 18px;"><b>{}</b></div>'.format(prompt)))
    printhtml(HTML(output))
 
 
  with open("cluster_centers.npy", "rb") as inp:
    cluster_centers = np.load(inp)
 
  with open("clickbait_vector.npy", "rb") as inp:
    clickbait_vector = np.load(inp)
  
  with io.capture_output() as captured:
    emb_model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1');
 
  print('\x1b[1m' + "Всё загружено, можно начинать проверять" + '\x1b[0m')
 
except:
  print('\x1b[1;31m' + "Что-то пошло не так. Пререзагрузите среду (Среда выполнения ➔ Сбросить настройки среды выполнения ➔ ДА)" + '\x1b[0m')

Всё загружено, можно начинать проверять


In [2]:
#@markdown ### Оценка на кликбейтность
#@markdown Введите ваш заголовок для проверки и вновь нажмите '▷'
 
Заголовок = "Морковь полезет как грибы! Посыпьте грунт простым советским..."  #@param {"type": "string"}
 
prettyprint_clickbait_score(Заголовок, model=emb_model, cluster_centers=cluster_centers, clickbait_vector=clickbait_vector)

## Жёлтые нейроновости

In [3]:
#@markdown  ### Подготовка генератора нейрокликбейта
#@markdown ← Нажмите '▷' слева, чтобы подготовить модель генерации заголовков

import re
passed = True

try:
  type(get_embedding_clickbait_score)
except NameError:
  passed = False
  print('\x1b[1;31m' + "Для генерации необходимо инициализировать оценку на кликбейтность – пожалуйста, запустите код в самой верхней клетке и затем вернитесь к этой." + '\x1b[0m')

if passed:
  report = !nvidia-smi -q --display=MEMORY
  report = " ".join(list(report))
  try:
    GiB = max([float(q) for q in re.findall(r"([0-9]+) MiB", report)])
    if GiB <= 14900:
      passed = False
      print('\x1b[1;31m' + "Подходящая видеокарта недоступна – перезагрузите среду (Среда выполнения ➔ Сбросить настройки среды выполнения ➔ ДА) и заново выполните подготовку оценки кликбейтности" + '\x1b[0m')
  except Exception as e:
    passed = False
    print('\x1b[1;31m' + "Что-то пошло не так. Проверьте, подключены ли вы к среде с GPU (Среда выполнения ➔ Сменить среду выполнения ➔ Аппаратный ускоритель ➔ GPU) и заново выполните подготовку оценки кликбейтности" + '\x1b[0m')
 
if passed:
  !pip install -qqq transformers==3.5.0 torch==1.6.0 1>/dev/null 2>/dev/null
  !pip install -qqq aitextgen 1>/dev/null 2>/dev/null
  !wget -q https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
 
  import logging
  logging.basicConfig(
          format="%(asctime)s — %(levelname)s — %(name)s — %(message)s",
          datefmt="%m/%d/%Y %H:%M:%S",
          level=logging.INFO
      )
 
  for model in ["aititles", "neuroclickbait"]:
    !wget -q https://storage.googleapis.com/isikus/machine-scorning/{model}.tar.gz
    !mkdir {model}
    !tar -xzf {model}.tar.gz -C {model}
  
  
  import json
  import os
  import sys
  import json
  import random
  
  from contextlib import contextmanager
  from datetime import datetime
  from IPython.display import Markdown
  from IPython.display import display as displaymd
  from aitextgen import aitextgen
  

  @contextmanager
  def all_logging_disabled(highest_level=logging.CRITICAL):
    previous_level = logging.root.manager.disable
    logging.disable(highest_level)
    
    try:
      yield
    finally:
      logging.disable(previous_level)
  
  def printmd(instr):
    displaymd(Markdown(instr))
  
  def get_lbc(model_meta):
    lbc = model_meta["line_break_character"]
    lbc = lbc.replace("\s", " ")
    return lbc
  
  def prettyscore(clickbait_score):
    if clickbait_score < 0:
      clickbait_score = 0
    elif clickbait_score > 100:
      clickbait_score = 100
    clickbait_score = int(round(clickbait_score))
    return clickbait_score
 
 
  try:
    with open(model+"/meta.json", "r") as injson:
      model_meta = json.load(injson)
  except Exception as e:
    passed = False
    print('\x1b[1;31m'+"Что-то сломалось – модель не готова :("+'\x1b[0m')
 
if passed:
  with all_logging_disabled():
    ai = aitextgen(model_folder="aititles", config="aititles/config.json", to_gpu=True)
  
  stup_cases = {
    0: {"k": 5, "p": 0.95, "temp": 1.0},
    25: {"k": 5, "p": 0.93, "temp": 2.0},
    50: {"k": 6, "p": 0.91, "temp": 3.0},
    75: {"k": 6, "p": 0.9, "temp": 4.0},
    100: {"k": 7, "p": 0.88, "temp": 5.0}
  }
 
  print('\x1b[1m' + "Всё в порядке, можно приступать к генерации" + '\x1b[0m')

Всё в порядке, можно приступать к генерации


In [4]:
#@markdown ### Создайте 3 своих уникальных нейрокликбейтных текста! Или 6. Или 9. Или больше.
 
titletext = ai.generate_one(max_length=1024, prompt="<s>")
titles = [t for t in re.findall(r"<s>(.*?)</s>", titletext) if t]
scores = [get_embedding_clickbait_score(e, cluster_centers=cluster_centers, clickbait_vector=clickbait_vector) for e in emb_model.encode(titles, show_progress_bar=False)]
best_scores = list(reversed(sorted(scores)))[:3]
best_titles = [titles[scores.index(s)] for s in best_scores]
best_scores = [str(prettyscore(s)) for s in best_scores]
 
s = ""
 
prompts = best_titles
 
for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * 1
 
with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)
 
rp = str(model_meta['repetition_penalty']) if "repetition_penalty" in model_meta else "1.0"
 
Бредовость = 0 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость
 
sd = random.choice(list(range(0, 10000)))
 
responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --repetition_penalty={rp} \
    --seed={str(sd)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null
 
responses = "\n".join(list(responses))
 
lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])
 
resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
sps = [sp for sp in s.split("\n") if sp]
for i, pair in enumerate(zip(sps, resps)):
  sp, resp = pair
  pt = re.search(re.sub(r"%[a-z]+", "(.*)", re.escape(model_meta["prompt"])), sp).group(1)
  rs, _ = re.findall(re.compile(re.escape(sp) + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("<b>" + p + "</b>" for p in re.split(lbc, pt) if p)
  pt += " (кликбейтность: " + "<i>" + best_scores[i] + "</i>" + ")"
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  rse = len(rs) if rs.find("Context >>") == -1 else rs.find("Context >>")
  printmd(rs[:rse] + "<br>")

<b>Собчак призналась в употреблении известной ведущей</b> (кликбейтность: <i>100</i>)

Ксения Собчак опубликовала на своей странице в Instagram фотографию, сделанную во время съемок программы "Пусть говорят". На снимке она предстала перед зрителями с букетом цветов. В кадре знаменитость позировала вместе со съемочной группой шоу. Поклонники отметили ее эффектную фигуру и стройный станок телеведущей: "Выглядите великолепно", "Ксюша вы супер! Вы просто красотка!", "Супер!!!" Напомним, что ранее стало известно о том, что у Ксении Собченковой обнаружили рак мозга из-за употребления алкоголя после участия в телешоу "Дом 2". После этого актриса обратилась к врачам за помощью и удалила себе опухоль. Однако врачи не нашли злокачественного новообразования. Отметим, что ведущая передачи "Давай поженимся!" появилась на программе еще несколько месяцев назад. Тогда ей диагностировали меланому – метастазы были очень серьезными для здоровья теледивы. Сейчас звезда активно готовится ко второму приему химиотерапии, чтобы избежать рецидива болезни.<br>

<b>Атакова самая красивая и мороженая заключка. Что скрывается и как его сделать</b> (кликбейтность: <i>100</i>)

Сегодня, 8 ноября, на канале "Сторис" выйдет новый сезон сериала «Атаки». На этот раз в нем будет играть актриса Анастасия Волочкова. Актриса рассказала о съемках нового сезона. В первом сезоне зрители увидели Анастасию Волочкову с ее сыном Артемом (настоящее имя — Александр). Съемки проходили во Владимирской области: там у балерины были съемки. Но уже сейчас она призналась, что очень любит мороженое. По словам актрисы, когда-то ей приходилось есть мороженное из банки или же просто пить воду со льдом. После этого она решила отказаться от него полностью – даже не задумываясь над тем фактом, что это вредно для здоровья организма! Сейчас Анастасия продолжает работать вместе со своей дочкой Ариадной в спектакле Театра им. Ермоловой по пьесе Островского. Также артистка занимается воспитанием сына Платона, которому скоро исполнится годик. Как отметила Настя, пока еще нет уверенности, получится ли у нее получить образование в школе или поступить куда угодно. Однако если все пойдет хорошо… то можно смело говорить, что сериал действительно интересный. Кстати сказать, после съемок в новом проекте поклонники звезды могут увидеть актрису совсем другими глазами, нежели прежде. Напомним, что в сериале снялись многие известные актеры - например, Андрей Миронов, Михаил Боярский... Теперь они играют главные роли в новых сериях проекта «Атаки» под названием «Каменская», а также принимают участие в различных театральных постановках.<br>

<b>Располневшая Кудрявцева обеспокоена сорачениями</b> (кликбейтность: <i>97</i>)

Певица Ольга Бузова опубликовала снимок с мужем Максимом Галкиным. На нем звезда позирует в бикини, а также демонстрирует подтянутую фигуру и стройные бедра, сообщает "7Дней". Подписчики обратили внимание на то обстоятельство, что телеведущая не может скрыть своего лишнего веса. Они предположили, что у нее проблемы со здоровьем. Поклонники Ольги считают ее очень худой – она часто сидит за компьютером или просто занимается спортом. Некоторые подписчики заметили изменения во внешности звезды шоу-бизнеса: они стали худыми, как после пластики, но при этом сохранили пышную грудь и упругие ягодицы (орфография и пунктуация сохранены). Другие отметили сходство между Ольгой и супругом артистки Александром Реввой. Многие поклонники певицы уверены, что певица сильно располнела из-за стресса. Напомним, ранее сообщалось о том, что бывшая возлюбленная футболиста Дениса Вороненкова Виктория Тарасова стала жертвой сексуальных домогательств бывшего мужа. Она обвинила его в домашнем насилии. По мнению Виктории Тарасовы, экс-супруг постоянно провоцировал свою супругу на интимные отношения. Однако та отказалась от этих обвинений и продолжила жить своей обычной жизнью без мужчин. Отметим, сейчас исполнительница находится дома под присмотром врачей, так как чувствует себя хорошо. 
<br>